In [ ]:
# =============================================================================
#           03_dinov2_train.ipynb: DINOv2 纯视觉模型完整训练脚本
# =============================================================================

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tqdm.notebook import tqdm
from PIL import Image
import os

# -----------------------------------------------------------------------------
# 1. 配置参数与设备
# -----------------------------------------------------------------------------
IMAGE_DIR = 'csiro-biomass'           # 图片根目录
TRAIN_CSV = 'csiro-biomass/train.csv'    # 训练标签文件
BATCH_SIZE = 16            # 批次大小 (显存不足可调小)
NUM_EPOCHS = 20            # 训练轮数
LR = 1e-3                  # 学习率
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(f"正在使用设备: {device}")

# -----------------------------------------------------------------------------
# 2. 数据准备：读取并转换为宽格式
# -----------------------------------------------------------------------------
print("正在处理数据...")
# 读取原始数据
df = pd.read_csv(TRAIN_CSV)

# 使用 pivot_table 将长格式 (Long) 转换为宽格式 (Wide)
# 这样每张图片对应一行，包含 5 个目标值
df_wide = pd.pivot_table(df, 
                         index=['image_path'], 
                         columns='target_name', 
                         values='target',
                         aggfunc='mean').reset_index()

# 目标列名列表
target_cols = ['Dry_Clover_g', 'Dry_Dead_g', 'Dry_Green_g', 'GDM_g', 'Dry_Total_g']

# 划分训练集和验证集 (80% / 20%)
train_df, val_df = train_test_split(df_wide, test_size=0.2, random_state=42)
print(f"数据准备完成。训练集: {len(train_df)} 张, 验证集: {len(val_df)} 张")


# -----------------------------------------------------------------------------
# 3. 定义类：数据集与模型
# -----------------------------------------------------------------------------

class BiomassDataset(Dataset):
    def __init__(self, dataframe, image_dir, target_cols, transform=None):
        self.df = dataframe
        self.image_dir = image_dir
        self.transform = transform
        # 预先进行 Log1p 变换，使目标分布更平滑
        self.targets = np.log1p(self.df[target_cols].values.astype(np.float32))

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path_rel = self.df.iloc[idx]['image_path']
        img_path = os.path.join(self.image_dir, img_path_rel)
        
        # 打开图片
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
            
        target = torch.tensor(self.targets[idx], dtype=torch.float)
        return image, target

class DINOv2VisualModel(nn.Module):
    def __init__(self, num_targets=5):
        super(DINOv2VisualModel, self).__init__()
        
        print("正在加载 DINOv2 (Small) 模型，首次运行会自动下载...")
        # 加载 Facebook 的 DINOv2 Small 模型
        self.backbone = torch.hub.load('facebookresearch/dinov2', 'dinov2_vits14')
        
        # 冻结骨干网络 (Backbone)，因为 DINOv2 特征已经非常强大
        for param in self.backbone.parameters():
            param.requires_grad = False
            
        # DINOv2 Small 输出维度是 384
        self.embed_dim = 384 
        
        # 定义回归头 (Head)
        self.head = nn.Sequential(
            nn.Linear(self.embed_dim, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_targets)
        )

    def forward(self, image):
        # 提取特征: 使用 DINOv2 的 [CLS] token
        features = self.backbone.forward_features(image)['x_norm_clstoken']
        # 预测
        output = self.head(features)
        return output

# -----------------------------------------------------------------------------
# 4. 初始化加载器与模型
# -----------------------------------------------------------------------------

# 定义图像变换 (DINOv2 需要 14 的倍数，推荐 224)
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(brightness=0.2, contrast=0.2), # 稍微增强
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# 实例化 Dataset 和 DataLoader
train_dataset = BiomassDataset(train_df.reset_index(drop=True), IMAGE_DIR, target_cols, transform=data_transforms['train'])
val_dataset = BiomassDataset(val_df.reset_index(drop=True), IMAGE_DIR, target_cols, transform=data_transforms['val'])

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

# 实例化模型
model = DINOv2VisualModel(num_targets=len(target_cols)).to(device)

# 定义损失函数和优化器
criterion = nn.MSELoss()
# 只优化 requires_grad=True 的参数 (即 Head 部分)
optimizer = optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=LR, weight_decay=1e-4)

# -----------------------------------------------------------------------------
# 5. 训练循环
# -----------------------------------------------------------------------------
best_rmse = float('inf')
save_path = 'best_dinov2_model.pth'

print(f"\n开始训练 DINOv2 模型 (共 {NUM_EPOCHS} 轮)...")

for epoch in range(NUM_EPOCHS):
    # --- 训练阶段 ---
    model.train()
    running_loss = 0.0
    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS} [训练]")
    
    for images, targets in pbar:
        images, targets = images.to(device), targets.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
    
    epoch_train_loss = running_loss / len(train_dataset)

    # --- 验证阶段 ---
    model.eval()
    all_preds = []
    all_targets = []
    
    with torch.no_grad():
        pbar_val = tqdm(val_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS} [验证]")
        for images, targets in pbar_val:
            images, targets = images.to(device), targets.to(device)
            outputs = model(images)
            
            # 还原 log 变换: expm1
            preds_orig = np.expm1(outputs.cpu().numpy())
            targets_orig = np.expm1(targets.cpu().numpy())
            
            all_preds.append(preds_orig)
            all_targets.append(targets_orig)
            
    # 计算验证集 RMSE
    val_rmse = np.sqrt(mean_squared_error(np.concatenate(all_targets), np.concatenate(all_preds)))
    
    print(f"Epoch {epoch+1}/{NUM_EPOCHS} -> 训练损失: {epoch_train_loss:.4f} | 验证 RMSE: {val_rmse:.4f}")
    
    # 保存最佳模型
    if val_rmse < best_rmse:
        best_rmse = val_rmse
        torch.save(model.state_dict(), save_path)
        print(f"  -> 🏆 新的最佳模型已保存: {save_path} (RMSE: {best_rmse:.4f})")

print(f"\n--- 训练完成 ---\n最好的 DINOv2 模型 RMSE 是: {best_rmse:.4f}")
